## **Import Dependencies**

In [1]:
from lxml import etree
import yaml
import pandas as pd
import optuna
import os
import simplace
from tqdm import tqdm
from glob import glob
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [5]:
import jpype

if jpype.isJVMStarted():
    print("JVM is running")
else:
    print("JVM is not running")

JVM is running


In [3]:
# Define directories
install_dir = r'simplace_portable/workspace/'
work_dir = r'simulation-optuna/'
output_dir = r'simulation-optuna/output/'

# Initialize Simplace
sp = simplace.initSimplace(install_dir, work_dir, output_dir, javaParameters=["-Xms256m"])

# Specify solution file
solution_file = r'simulation-optuna/SimulationExperimentTemplateTest/solution/Soil3_Germany_AllKreis_Test.sol.xml'
project_file = r'simulation-optuna/SimulationExperimentTemplateTest/project/Soil3_Germany_AllKreis.proj.xml'
simplace.openProject(sp, solution_file, project_file)
simplace.runProject(sp)

In [4]:
simplace.closeProject(sp)

In [ ]:
if jpype.isJVMStarted():
    jpype.shutdownJVM()

In [3]:
# Function to update single-value parameters (e.g., TBASEM)
def update_single_value_param(root, param_id, crop_name, new_value):
    xpath = f"//crop[parameter[@id='CropName' and text()='{crop_name}']]/parameter[@id='{param_id}']"
    for param in root.xpath(xpath):
        if not param.getchildren():
            param.text = str(new_value)

In [4]:
# Function to update multi-value parameters (e.g., PHOTTB)
def update_multi_value_param(root, param_id, crop_name, new_values):
    xpath = f"//crop[parameter[@id='CropName' and text()='{crop_name}']]/parameter[@id='{param_id}']"
    for param in root.xpath(xpath):
        value_elements = param.findall('value')
        for i, val in enumerate(new_values):
            if i < len(value_elements):
                value_elements[i].text = str(val)
            else:
                new_elem = etree.SubElement(param, 'value')
                new_elem.text = str(val)
        # # Remove any extra existing <value> tags
        for extra in value_elements[len(new_values):]:
            param.remove(extra)

In [5]:
# Define the XML path
xml_path = r'simulation-optuna\SimulationExperimentTemplateTest\data\crop\crop_cka_latest_USL_test.xml'
config_path = r'config.yaml'

In [6]:
def process_result(sim_dir, obs_dir):
    # Define the state name
    state = 'Brandenburg'

    # Read the location csv
    location_data = pd.read_csv(os.path.join('DE_DWD_UBN_Crop.csv'))
    location_data = location_data[['Cell_ID', 'NUTS_ID', 'NUTS_NAME', 'STATE_ID', 'STATE_NAME']]
    location_data.rename(columns={'Cell_ID': 'Location'}, inplace=True)
    location_data = location_data[location_data['STATE_NAME']==state]

    # Read the simulated data
    sim_data = pd.DataFrame()

    for path in tqdm(glob(os.path.join(sim_dir, '*.csv'))):
        df = pd.read_csv(path, delimiter=';')
        sim_data = pd.concat((sim_data, df), ignore_index=True)

    # Filter columns
    sim_variables = ['projectid', 'Year', 'AnthesisDOY', 'MaturityDOY', 'Yield_t_ha']
    sim_data_filtered = sim_data[sim_variables]
    sim_data_filtered.rename(columns={'projectid': 'Location'}, inplace=True)
    sim_data_filtered = pd.merge(
        left=sim_data_filtered, 
        right=location_data,
        on='Location', 
        how='inner'
    )

    # Read the observed datasets
    obs_phen_data = pd.read_csv(os.path.join(obs_dir))
    obs_phen_data['AnthesisDOY'] = pd.to_datetime(obs_phen_data['Flowering_DOY']).dt.day_of_year
    obs_phen_data['MaturityDOY'] = pd.to_datetime(obs_phen_data['Harvest_DOY']).dt.day_of_year
    obs_phen_data = obs_phen_data[['STATE_ID', 'STATE_NAME', 'Year', 'AnthesisDOY', 'MaturityDOY']]
    obs_phen_data = obs_phen_data[obs_phen_data['STATE_NAME']==state]

    sim_data_grouped = sim_data_filtered[['Year', 'AnthesisDOY', 'MaturityDOY']].groupby(by='Year')
    sim_data_grouped_mean = sim_data_grouped.mean().reset_index()
    sim_data_grouped_std = sim_data_grouped.std().reset_index()
    sim_data_grouped_mean[['AnthesisDOY', 'MaturityDOY']] = sim_data_grouped_mean[['AnthesisDOY', 'MaturityDOY']].astype('int')
    sim_data_grouped_std = sim_data_grouped_std[['Year', 'AnthesisDOY', 'MaturityDOY']].rename(
        columns={'AnthesisDOY': 'AnthesisDOY_std', 'MaturityDOY': 'MaturityDOY_std'}
    )
    sim_data_grouped_mean = pd.merge(left=sim_data_grouped_mean, right=sim_data_grouped_std, on='Year', how='inner')
    sim_data_grouped_mean.rename(
        columns={col: f'{col}_sim' for col in sim_data_grouped_mean.columns[1:]},
        inplace=True
    )

    phen_comparison_df = pd.merge(
    left=obs_phen_data[obs_phen_data['STATE_NAME']==state], 
    right=sim_data_grouped_mean, 
    on='Year', 
    how='inner'
    )

    return phen_comparison_df

In [7]:
# process_result(
#     r'C:\HALDER\GITHUB\MSM-Research\Simplace-Python\simulation-optuna\output\yearly',
#     r'C:\HALDER\GITHUB\MSM-Research\Simplace-Python\phenology_WW_1999_2021.csv'
# )

In [7]:
def objective(trial, xml_path, config_path, sim_dir, obs_dir):
    # Load XML fresh every time
    tree = etree.parse(xml_path)
    root = tree.getroot()

    # Load YAML config
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)

    crop_name = config['crop_name']
    single_value_specs = config.get('single_value_params', {})
    multi_value_specs = config.get('multi_value_params', {})

    # Suggest single value parameters
    for param_id, spec in single_value_specs.items():
        if spec['type'] == 'int':
            val = trial.suggest_int(param_id, spec['low'], spec['high'])
        elif spec['type'] == 'float':
            precision = spec.get('precision', 5)
            val = round(trial.suggest_float(param_id, spec['low'], spec['high']), precision)

        print(param_id, val)
        update_single_value_param(root, param_id, crop_name, val)

    # Suggest multi-value parameters
    for param_id, spec in multi_value_specs.items():
        values = []
        
        if spec['type'] == 'int':
            for i, bounds in enumerate(spec['values']):
                val = trial.suggest_int(f"{param_id}_{i}", bounds['low'], bounds['high'])
                values.append(val)

            update_multi_value_param(root, param_id, crop_name, values)
            print(param_id, values)
            
        elif spec['type'] == 'float':
            precision = spec.get('precision', 3)
            for i, bounds in enumerate(spec['values']):
                val = round(trial.suggest_float(f"{param_id}_{i}", bounds['low'], bounds['high']), precision)
                values.append(val)

            update_multi_value_param(root, param_id, crop_name, values)
            print(param_id, values)

    # Save modified XML
    tree.write(xml_path, pretty_print=True, xml_declaration=True, encoding='UTF-8')

    # Define directories
    install_dir = r'simplace_portable/workspace/'
    work_dir = r'simulation-optuna/'
    output_dir = r'simulation-optuna/output/'

    # Initialize Simplace
    sp = simplace.initSimplace(install_dir, work_dir, output_dir)

    # Specify solution file
    solution_file = r'simulation-optuna/SimulationExperimentTemplateTest/solution/Soil3_Germany_AllKreis_Test.sol.xml'
    project_file = r'simulation-optuna/SimulationExperimentTemplateTest/project/Soil3_Germany_AllKreis.proj.xml'
    simplace.openProject(sp, solution_file, project_file)

    # Run your simulation model here and read the result
    simplace.runProject(sp)
    simplace.closeProject(sp)

    result = process_result(sim_dir, obs_dir)

    rmse = np.sqrt(mean_squared_error(result['AnthesisDOY'], result['AnthesisDOY_sim']))

    return rmse

JVM is running


In [8]:
sim_dir = r'simulation-optuna\output\yearly'
obs_dir = r'phenology_WW_1999_2021.csv'

In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, xml_path, config_path, sim_dir, obs_dir), n_trials=5)

[I 2025-05-22 11:37:31,902] A new study created in memory with name: no-name-1f294d12-c8e7-42ee-bf6b-b182ba8dbb6f


TEFFMX 30.76
TSUM1 2209
TSUM2 424
RUETableRUE [3.11, 3.49, 3.07, 3.54]


100%|██████████| 3/3 [00:00<00:00, 402.58it/s]
[I 2025-05-22 11:38:59,385] Trial 0 finished with value: 52.06851673307593 and parameters: {'TEFFMX': 30.760197401379003, 'TSUM1': 2209, 'TSUM2': 424, 'RUETableRUE_0': 3.1133122515133724, 'RUETableRUE_1': 3.491353665540375, 'RUETableRUE_2': 3.0660558273650462, 'RUETableRUE_3': 3.5394795334390006}. Best is trial 0 with value: 52.06851673307593.
[W 2025-05-22 11:38:59,395] Trial 1 failed with parameters: {'TEFFMX': 38.4527594146807, 'TSUM1': 2310, 'TSUM2': 436, 'RUETableRUE_0': 3.2811137207652967, 'RUETableRUE_1': 3.6255016259379897, 'RUETableRUE_2': 3.1529898989697736, 'RUETableRUE_3': 3.940448014640243} because of the following error: OSError('JVM is already started').
Traceback (most recent call last):
  File "c:\Users\halder\AppData\Local\anaconda3\envs\py312\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\halder\AppData\Local\Temp\

TEFFMX 38.45
TSUM1 2310
TSUM2 436
RUETableRUE [3.28, 3.63, 3.15, 3.94]


OSError: JVM is already started

In [ ]:
# Class
import yaml
from lxml import etree
import optuna

class SimplaceOptimizer:
    def __init__(self, xml_path, config_path):
        self.xml_path = xml_path
        self.config_path = config_path
        self.config = self._load_config()

    def _load_config(self):
        with open(self.config_path, 'r') as f:
            return yaml.safe_load(f)

    def _load_xml(self):
        tree = etree.parse(self.xml_path)
        return tree, tree.getroot()

    def _suggest_single_params(self, trial, root):
        crop_name = self.config['crop_name']
        single_specs = self.config.get('single_value_params', {})

        for param_id, spec in single_specs.items():
            if spec['type'] == 'int':
                val = trial.suggest_int(param_id, spec['low'], spec['high'])
            elif spec['type'] == 'float':
                precision = spec.get('precision', 5)
                val = round(trial.suggest_float(param_id, spec['low'], spec['high']), precision)

            update_single_value_param(root, param_id, crop_name, val)

    def _suggest_multi_params(self, trial, root):
        crop_name = self.config['crop_name']
        multi_specs = self.config.get('multi_value_params', {})

        for param_id, spec in multi_specs.items():
            values = []
            if spec['type'] == 'int':
                for i, bounds in enumerate(spec['values']):
                    val = trial.suggest_int(f"{param_id}_{i}", bounds['low'], bounds['high'])
                    values.append(val)
            elif spec['type'] == 'float':
                precision = spec.get('precision', 3)
                for i, bounds in enumerate(spec['values']):
                    val = round(trial.suggest_float(f"{param_id}_{i}", bounds['low'], bounds['high']), precision)
                    values.append(val)

            update_multi_value_param(root, param_id, crop_name, values)

    def objective(self, trial):
        tree, root = self._load_xml()
        self._suggest_single_params(trial, root)
        self._suggest_multi_params(trial, root)

        # Save updated XML
        tree.write(self.xml_path, pretty_print=True, xml_declaration=True, encoding='UTF-8')

        # Run model and calculate score
        # simulated_yield = run_simulation(self.xml_path)
        # score = evaluate(simulated_yield)
        score = trial.suggest_float("dummy_score", 0.7, 0.9)  # Replace with real score
        return score

In [46]:
optimizer = SimplaceOptimizer(xml_path, config_path)
study = optuna.create_study(direction="minimize")
study.optimize(optimizer.objective, n_trials=10)

[I 2025-05-22 02:11:20,052] A new study created in memory with name: no-name-d157e040-706c-4b7b-a223-eafdf5d4f700
[I 2025-05-22 02:11:20,074] Trial 0 finished with value: 0.7668463420921908 and parameters: {'TEFFMX': 39.66944619132069, 'TSUM1': 1682, 'TSUM2': 674, 'RUETableRUE_0': 2.664652745162674, 'RUETableRUE_1': 3.05148158148924, 'RUETableRUE_2': 3.341120846449469, 'RUETableRUE_3': 3.6887364731616197, 'dummy_score': 0.7668463420921908}. Best is trial 0 with value: 0.7668463420921908.
[I 2025-05-22 02:11:20,090] Trial 1 finished with value: 0.7973347957356189 and parameters: {'TEFFMX': 39.78551345243318, 'TSUM1': 2113, 'TSUM2': 459, 'RUETableRUE_0': 2.805025805893568, 'RUETableRUE_1': 3.324112052559183, 'RUETableRUE_2': 3.99324345378398, 'RUETableRUE_3': 3.5892865208038858, 'dummy_score': 0.7973347957356189}. Best is trial 0 with value: 0.7668463420921908.
[I 2025-05-22 02:11:20,105] Trial 2 finished with value: 0.7501892594061048 and parameters: {'TEFFMX': 32.153729912735834, 'TSUM